# Fine-tuning Embeddings for RAG on Specific Data

As we start our "fine-tuning" week, we'll start with the lowest hanging improvement one can do for RAG - which is:

Fine-tuning embeddings!

- 🤝 Breakout Room #1:
  - Task 1: Dependencies and Boilerplate
  - Task 2: Loading Data
  - Task 3: Constructing a Fine-tuning Dataset
  - Task 4: Fine-tuning `snowflake-arctic-embed-m`
  - Task 5: Evaluating our Retriever

- 🤝 Breakout Room #2:
  - Task 1: Vibe Checking Our LCEL RAG Chain
  - Task 2: Ragas Evaluation



#### Basic Overview of Fine-tuning Embeddings

In essence, what we want to do when we fine-tune our embedding models is very simple:

```
Move the embeddings for questions relating to a document
closer together with that document
```

We can think of fine-tuning our embedding models as follows:

1) We have some pair of text items that *should* be closer together
  - `Question`, `Document` pairs
  - EX: `Who drives the bus?`, `The bus was driven by Kyle, the Bus Driver`.

2) We use these pairs as labeled data to fine-tune our embedding model.

The process of training helps the model more accurately associate our questions with the correct documents.

#####❓ Question #1:

Describe the nuance between using Q&D pairs to train the embedding model vs. inter-document pairs/related sentences.

What caveats does this approach have? Are there any special considerations for what kind of Q's we should use?

---

**ANSWER:**

We are specifically relating *the questions* to *the documents*. This means that we are making our embedding model at the very specific task of relating potential questions to specific documents.

There are many caveats, but the main ones are:

- Your Q's should reflect the Q's of your users
- This kind of fine-tuning will (purposefully) "overfit" on your data; this is the desired result in this case.

## Task 1: Dependencies and Boilerplate

We'll set up our `nest_asyncio` so we can leverage async loops in our Notebook.

We'll also install the required libraries we'll be using today, and set up our OpenAI API key!

### Nest Asyncio

In [1]:
import nest_asyncio

nest_asyncio.apply()

### Install Dependencies

In [2]:
!pip install -qU langchain_openai langchain_huggingface langchain_core==0.2.38 langchain langchain_community langchain-text-splitters

In [3]:
!pip install -qU faiss-cpu unstructured==0.15.7 python-pptx==1.0.2 nltk==3.9.1

### Provide OpenAI API Key

In [4]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key: ")

## Task 2: Loading Data

We'll be using a recent document released by the EU 'laying down harmonised rules on artificial intelligence and amending Regulations'.

The data can be found [here](https://eur-lex.europa.eu/legal-content/EN/TXT/?uri=CELEX%3A32024R1689), though we will be using a HTML version which was collected into the AIM DataRepository.

First, we'll clone and then `cd` into the DataRepository.

In [5]:
!git clone https://github.com/AI-Maker-Space/DataRepository.git

fatal: destination path 'DataRepository' already exists and is not an empty directory.


In [6]:
%cd DataRepository

/Users/david/Desktop/aie4/Week 5/Day 1/DataRepository


Next we're going to be using the `UnstructuredHTMLLoader` to load our HTML document into a LangChain document using the [Unstructured](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.html.UnstructuredHTMLLoader.html) library.

In [7]:
from langchain_community.document_loaders import UnstructuredHTMLLoader

training_documents_loaded = UnstructuredHTMLLoader("eu_ai_act.html")

Next, we'll set up a classic naive chunking strategy as we only care that the documents get parsed into chunks that we can generate synthetic questions about.

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 750,
    chunk_overlap  = 20,
    length_function = len
)

Next we can load/split these documents as follows.

In [9]:
training_documents = text_splitter.split_documents(training_documents_loaded.load())


In [10]:
# Inspect one of the the documents to understand it's stucture
training_documents[0]

Document(metadata={'source': 'eu_ai_act.html'}, page_content='Official Journal of the European Union EN L series\n\n2024/1689 12.7.2024\n\nREGULATION (EU) 2024/1689 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\n\nof 13 June 2024\n\nlaying down harmonised rules on artificial intelligence and amending Regulations (EC) No 300/2008, (EU) No 167/2013, (EU) No 168/2013, (EU) 2018/858, (EU) 2018/1139 and (EU) 2019/2144 and Directives 2014/90/EU, (EU) 2016/797 and (EU) 2020/1828 (Artificial Intelligence Act)\n\n(Text with EEA relevance)\n\nTHE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE EUROPEAN UNION,\n\nHaving regard to the Treaty on the Functioning of the European Union, and in particular Articles 16 and 114 thereof,\n\nHaving regard to the proposal from the European Commission,')

In [11]:
len(training_documents)

1029

Next, we're going to associate each of our chunks with a unique identifier.

In [12]:
import uuid

id_set = set()

# Assign each chunk a unique identifier
for document in training_documents:
  id = str(uuid.uuid4())

  # Check for uniqueness
  while id in id_set:
    id = uuid.uuid4()

  # Add the id to the set
  id_set.add(id)

  # Add the id to the document's metadata
  document.metadata["id"] = id

Next, we'll simply use naive Python slicing to create a training, test, and validation set to prepare our data for the next step.

In [13]:
training_split_documents = training_documents[:300]
val_split_documents = training_documents[300:350]
test_split_documents = training_documents[350:400]

## Task 3: Constructing a Fine-tuning Dataset

Using the nodes we created above, we can finally start constructing a fine-tuning dataset utilizing OpenAI's `gpt-4o-mini` (released [July 18th](https://openai.com/index/gpt-4o-mini-advancing-cost-efficient-intelligence/)).

The basic idea here is straightforward enough:

1. We look at a document
2. We generate questions that could be answered by that node

This gives us a number of question/context pairs that we can use to fine-tune our Embeddings model.

In [14]:
from langchain_openai import ChatOpenAI

qa_chat_model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

We'll create a simple Question Generation prompt to query `gpt-4o-mini` to generate Questions for each retrieved context.

In [15]:
from langchain_core.prompts import ChatPromptTemplate

qa_prompt = """\
Given the following context, you must generate questions based on only the provided context.

You are to generate {n_questions} questions which should be provided in the following format:

1. QUESTION #1
2. QUESTION #2
...

Context:
{context}
"""

qa_prompt_template = ChatPromptTemplate.from_template(qa_prompt)

We'll create a simple chain to query the LLM!

In [16]:
question_generation_chain = qa_prompt_template | qa_chat_model

There's a lot going on in this function - let's take a deeper look:

1. First, we provide a list of documents and a number of questions
2. We, for each document in our list, generate `n_questions` of questions.
3. We then associate those questions and contexts via a `UUID`.

> NOTE: The reason we're doing this `UUID` association is for ease of use later in the notebook.

##### 🏗️ Activity #1:

We have:

- Lists of `Documents` with the `metadata` field `id`.

We need:

- An object with key `id`, which have values `str` questions.
- An object with key `question_id`, which have values `List(str)` which will be a list of associated `context_id`.

An Example:

question_object:
```python
{
'b4b95fb6-f827-4454-aa5b-20e62733f172': 'What types of accessible formats are available for persons with disabilities?',
'df58ee4f-714c-419e-8324-94e5870574e2': 'How do accessible formats benefit persons with disabilities?',
'505fce8b-0e56-48de-a251-61027e396918': 'What are some of the risks associated with the increasing capabilities of AI systems that generate synthetic content?',
'8ff0ab33-60dc-4fee-8958-91bfb686aca8': 'Why is it important for providers of AI systems to embed technical solutions for marking and detecting synthetic content?'
}
 ```

 context_object:
 ```python
{
'b4b95fb6-f827-4454-aa5b-20e62733f172': ['dd75bf94-75f3-4603-8e4b-5522f6925638'],
'df58ee4f-714c-419e-8324-94e5870574e2': ['dd75bf94-75f3-4603-8e4b-5522f6925638'],
'505fce8b-0e56-48de-a251-61027e396918': ['ffe3893f-688c-48e8-90bd-7a9feb953d90'],
'8ff0ab33-60dc-4fee-8958-91bfb686aca8': ['ffe3893f-688c-48e8-90bd-7a9feb953d90'],
}
 ```

 As you can see, a piece of context can be associated with more than 1 question.

 The task is to write the Python function(s) to accomplish this task.

 Your function signature is provided below, along with the desired return values.

 > NOTE: You can make any modifications that you desire - assuming that you have the correct input and outputs.

In [17]:
import tqdm

def create_questions(documents, n_questions):
  questions = {}
  relevant_docs = {}

  for document in tqdm.tqdm(documents):
    # Get the context and a empty list object for the generated questions
    document_content = {"context" : document.page_content, "questions" : []}

    # Generate the questions the chain - note we pass in as parameters the context and the number of questions we want generated
    questions_generated = question_generation_chain.invoke({"context": document.page_content, "n_questions": n_questions})

    # Inspect the response
    #print(questions_generated)

    #
    for question in questions_generated.content.split("\n"):
      # Generate a unique id
      question_id = str(uuid.uuid4())
      # Extract question text
      questions[question_id] = "".join(question.split(".")[1:]).strip()

      # Associate the related chunk
      relevant_docs[question_id] = [document.metadata["id"]]

  return questions, relevant_docs

We'll use the function to generate training, validation, and test data with `n_questions=2` for each.

In [18]:
training_questions, training_relevant_contexts = create_questions(training_split_documents, 2)

100%|██████████| 300/300 [03:28<00:00,  1.44it/s]


In [19]:
val_questions, val_relevant_contexts = create_questions(val_split_documents, 2)

100%|██████████| 50/50 [00:33<00:00,  1.48it/s]


In [20]:
test_questions, test_relevant_contexts = create_questions(test_split_documents, 2)

100%|██████████| 50/50 [00:38<00:00,  1.31it/s]


### Reformating and Saving Datasets

Now, we can save our datasets for later use!

> NOTE: If you ran into issues creating the data - you can use the data from the DataRespository. It's simply called: `train_dataset.jsonl`, etc.

In [21]:
import json

training_corpus = {train_item.metadata["id"] : train_item.page_content for train_item in training_split_documents}

train_dataset = {
    "questions" : training_questions,
    "relevant_contexts" : training_relevant_contexts,
    "corpus" : training_corpus
}

with open("training_dataset.jsonl", "w") as f:
  json.dump(train_dataset, f)

In [22]:
val_corpus = {val_item.metadata["id"] : val_item.page_content for val_item in val_split_documents}

val_dataset = {
    "questions" : val_questions,
    "relevant_contexts" : val_relevant_contexts,
    "corpus" : val_corpus
}

with open("val_dataset.jsonl", "w") as f:
  json.dump(val_dataset, f)

In [23]:
train_corpus = {test_item.metadata["id"] : test_item.page_content for test_item in test_split_documents}

test_dataset = {
    "questions" : test_questions,
    "relevant_contexts" : test_relevant_contexts,
    "corpus" : train_corpus
}

with open("test_dataset.jsonl", "w") as f:
  json.dump(test_dataset, f)

## Task 4: Fine-tuning `snowflake-arctic-embed-m`

Now that we have a dataset, let's grab a `sentence-transformers` Embeddings model!

We'll be using Snowflake's [`snowflake-arctic-embed-m`](https://huggingface.co/Snowflake/snowflake-arctic-embed-m) as a base embeddings model.

It is a well performing embeddings model by itself, but there's a lot of very specific domain terms and vocabulary in our courpus - so lets fine-tune it and see what that can do for us!

In [24]:
!pip install -qU sentence_transformers datasets pyarrow

In [25]:
from sentence_transformers import SentenceTransformer

model_id = "Snowflake/snowflake-arctic-embed-m"
model = SentenceTransformer(model_id)

/opt/anaconda3/envs/aie4/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


We'll grab some necessary imports from `sentence_transformers` and `torch`.

> NOTE: PyTorch (`torch`) is a popular machine learning library - while we don't go very deep into PyTorch it's an incredibly powerful and interesting library! Please read more about it [here](https://pytorch.org/tutorials/beginner/basics/intro.html)!

In [26]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sentence_transformers import InputExample

We're using a toy batch size here to reflect the limited number of examples we have.

> NOTE: It is typical to use a much larger batch size (~64+), hardware permitting.

In [27]:
BATCH_SIZE = 20

Let's move our dataset into the expected format for training.

In [28]:
corpus = train_dataset['corpus']
queries = train_dataset['questions']
relevant_docs = train_dataset['relevant_contexts']

examples = []
for query_id, query in queries.items():
    doc_id = relevant_docs[query_id][0]
    text = corpus[doc_id]
    example = InputExample(texts=[query, text])
    examples.append(example)

Now we can create a `torch` `DataLoader`!

In [29]:
loader = DataLoader(
    examples, batch_size=BATCH_SIZE
)

Next up, we'll prepare our loss function!

Loss is an important part of training, fine-tuning, and more. If you want a deep dive on loss - you can check out our [event on loss!](https://www.youtube.com/watch?v=iB8FWR9aD5Q&t=8s).

The core loss we're using today is called `MultipleNegativesRankingLoss` - you can find more information [here](https://github.com/UKPLab/sentence-transformers/blob/master/sentence_transformers/losses/MultipleNegativesRankingLoss.py).

This is "wrapped" in `MatryoshkaLoss`, which you can read the implementation of [here](https://github.com/UKPLab/sentence-transformers/blob/master/sentence_transformers/losses/MatryoshkaLoss.py).

In [30]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

matryoshka_dimensions = [768, 512, 256, 128, 64]
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

##### 🏗️ Activity #2:

Both of these losses sound "cool", but what are they - exactly - under the hood?

Why are these losses specifically doing? Please write a short summary of each loss.

> NOTE: This is a course focused on AI Engineering and the application of AI - looking for a hint? Try pasting the code (linked above) into ChatGPT/Claude to write the summary!

***\*ANSWER\****

From Gemini:  

*MatryoshkaLoss:*

*Purpose: This loss function is designed for training sentence transformers with hierarchical representations. It combines multiple inner loss functions at different levels of the model's hierarchy, allowing for more flexible and effective training.*

*Parameters:*

*model: The sentence transformer model being trained.*

*inner_train_loss: The inner loss function to be used at each level of the hierarchy. In this case, it's MultipleNegativesRankingLoss.*

*matryoshka_dims: A list of integers specifying the dimensions of the hidden layers in the hierarchical model.*

*Functionality:*

*The MatryoshkaLoss iterates through the hidden layers of the model from the deepest to the shallowest.*

*At each level, it applies the inner loss function (MultipleNegativesRankingLoss in this case) to the corresponding hidden representations.*

*The loss values from all levels are combined using a weighted average, with weights determined based on the hierarchical structure.*

*MultipleNegativesRankingLoss:*

*Purpose: This loss function is commonly used for training sentence transformers in tasks like semantic similarity or ranking. It aims to maximize the similarity between positive pairs (sentences with similar meanings) and minimize the similarity between positive and negative pairs (sentences with different meanings).*

*Parameters:*

*model: The sentence transformer model being trained.*

*Functionality:*

*Given a batch of sentences, the MultipleNegativesRankingLoss calculates the similarity between each positive pair and each negative pair using the model's output.*

*The loss is computed based on the ranking of positive and negative pairs. It aims to push positive pairs to higher ranks than negative pairs.*

*Overall:*

*In the provided code, the MatryoshkaLoss is used to train the sentence transformer model with a hierarchical structure. The MultipleNegativesRankingLoss serves as the inner loss function at each level of the hierarchy. This combination allows for more effective training by considering the model's representations at different levels and focusing on improving the ranking of positive and negative pairs.*



Now we can set-up our evaluator.

> NOTE: Due to the formatting of our dataset - this is all we have to do!

In [31]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

corpus = val_dataset['corpus']
queries = val_dataset['questions']
relevant_docs = val_dataset['relevant_contexts']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

We'll train this model for 5 epochs, though you could increase this number if we had a significant amount more data.

In [32]:
EPOCHS = 5

It's training time!

> NOTE: We're manually defining a warm-up period here - this is just to provide a smooth ramp into our training!

In [33]:
!pip install accelerate -U

In [34]:
warmup_steps = int(len(loader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(loader, train_loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path='finetuned_arctic',
    show_progress_bar=True,
    evaluator=evaluator,
    evaluation_steps=50,
)

 33%|███▎      | 50/150 [01:00<02:25,  1.46s/it]

{'eval_cosine_accuracy@1': 0.88, 'eval_cosine_accuracy@3': 0.97, 'eval_cosine_accuracy@5': 0.99, 'eval_cosine_accuracy@10': 1.0, 'eval_cosine_precision@1': 0.88, 'eval_cosine_precision@3': 0.3233333333333333, 'eval_cosine_precision@5': 0.19799999999999998, 'eval_cosine_precision@10': 0.09999999999999998, 'eval_cosine_recall@1': 0.88, 'eval_cosine_recall@3': 0.97, 'eval_cosine_recall@5': 0.99, 'eval_cosine_recall@10': 1.0, 'eval_cosine_ndcg@10': 0.9450313882468356, 'eval_cosine_mrr@10': 0.9266666666666667, 'eval_cosine_map@100': 0.9266666666666667, 'eval_dot_accuracy@1': 0.88, 'eval_dot_accuracy@3': 0.97, 'eval_dot_accuracy@5': 0.99, 'eval_dot_accuracy@10': 1.0, 'eval_dot_precision@1': 0.88, 'eval_dot_precision@3': 0.3233333333333333, 'eval_dot_precision@5': 0.19799999999999998, 'eval_dot_precision@10': 0.09999999999999998, 'eval_dot_recall@1': 0.88, 'eval_dot_recall@3': 0.97, 'eval_dot_recall@5': 0.99, 'eval_dot_recall@10': 1.0, 'eval_dot_ndcg@10': 0.9450313882468356, 'eval_dot_mrr@10'

 67%|██████▋   | 100/150 [02:03<01:07,  1.34s/it]

{'eval_cosine_accuracy@1': 0.88, 'eval_cosine_accuracy@3': 0.97, 'eval_cosine_accuracy@5': 0.99, 'eval_cosine_accuracy@10': 1.0, 'eval_cosine_precision@1': 0.88, 'eval_cosine_precision@3': 0.3233333333333333, 'eval_cosine_precision@5': 0.19799999999999998, 'eval_cosine_precision@10': 0.09999999999999998, 'eval_cosine_recall@1': 0.88, 'eval_cosine_recall@3': 0.97, 'eval_cosine_recall@5': 0.99, 'eval_cosine_recall@10': 1.0, 'eval_cosine_ndcg@10': 0.9459024482741616, 'eval_cosine_mrr@10': 0.9278333333333333, 'eval_cosine_map@100': 0.9278333333333333, 'eval_dot_accuracy@1': 0.88, 'eval_dot_accuracy@3': 0.97, 'eval_dot_accuracy@5': 0.99, 'eval_dot_accuracy@10': 1.0, 'eval_dot_precision@1': 0.88, 'eval_dot_precision@3': 0.3233333333333333, 'eval_dot_precision@5': 0.19799999999999998, 'eval_dot_precision@10': 0.09999999999999998, 'eval_dot_recall@1': 0.88, 'eval_dot_recall@3': 0.97, 'eval_dot_recall@5': 0.99, 'eval_dot_recall@10': 1.0, 'eval_dot_ndcg@10': 0.9459024482741616, 'eval_dot_mrr@10'

100%|██████████| 150/150 [03:09<00:00,  1.06s/it]

{'eval_cosine_accuracy@1': 0.87, 'eval_cosine_accuracy@3': 0.97, 'eval_cosine_accuracy@5': 0.98, 'eval_cosine_accuracy@10': 1.0, 'eval_cosine_precision@1': 0.87, 'eval_cosine_precision@3': 0.3233333333333333, 'eval_cosine_precision@5': 0.19599999999999995, 'eval_cosine_precision@10': 0.09999999999999998, 'eval_cosine_recall@1': 0.87, 'eval_cosine_recall@3': 0.97, 'eval_cosine_recall@5': 0.98, 'eval_cosine_recall@10': 1.0, 'eval_cosine_ndcg@10': 0.941905289608611, 'eval_cosine_mrr@10': 0.9225, 'eval_cosine_map@100': 0.9225, 'eval_dot_accuracy@1': 0.87, 'eval_dot_accuracy@3': 0.97, 'eval_dot_accuracy@5': 0.98, 'eval_dot_accuracy@10': 1.0, 'eval_dot_precision@1': 0.87, 'eval_dot_precision@3': 0.3233333333333333, 'eval_dot_precision@5': 0.19599999999999995, 'eval_dot_precision@10': 0.09999999999999998, 'eval_dot_recall@1': 0.87, 'eval_dot_recall@3': 0.97, 'eval_dot_recall@5': 0.98, 'eval_dot_recall@10': 1.0, 'eval_dot_ndcg@10': 0.941905289608611, 'eval_dot_mrr@10': 0.9225, 'eval_dot_map@10

100%|██████████| 150/150 [03:10<00:00,  1.27s/it]

{'train_runtime': 190.6217, 'train_samples_per_second': 15.738, 'train_steps_per_second': 0.787, 'train_loss': 1.2155982462565105, 'epoch': 5.0}


## Task 5: Evaluating our Retriever

Now that we have fine-tuned our retriever - let's see if it's worthwhile!

We'll start with some basic imports.

In [35]:
import pandas as pd

from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document

Now we'll define a function that will help us evaluate our retrieval process.

> NOTE: We're assuming 1 correct document in a "hit".

In [36]:
# !pip install tqdm

from tqdm import tqdm


def evaluate_openai(
    dataset,
    embed_model,
    top_k=5,
    verbose=False,
):
  corpus = dataset['corpus']
  questions = dataset['questions']
  relevant_docs = dataset['relevant_contexts']
  documents = [Document(page_content=content, metadata={"id": doc_id}) for doc_id, content in corpus.items()]
  vectorstore = FAISS.from_documents(documents, embed_model)

  retriever = vectorstore.as_retriever(search_kwargs={"k": top_k})

  eval_results = []
  for id, question in tqdm(questions.items()):
    retrieved_nodes = retriever.invoke(question)
    retrieved_ids = [node.metadata["id"] for node in retrieved_nodes]
    expected_id = relevant_docs[id][0]
    is_hit = expected_id in retrieved_ids
    eval_results.append({"id": id, "question": question, "expected_id": expected_id, "is_hit": is_hit})

  return eval_results

All that's left to do is evaluate, we'll evaluate our model against:

1. OpenAI's closed source `text-embedding-3-small`
2. The base non-fine-tuned version of `Snowflake/snowflake-arctic-embed-m`.

Let's see how it stacks up!

### `text-embedding-3-small`

In [37]:
te3_openai = OpenAIEmbeddings(model="text-embedding-3-small")
te3_results = evaluate_openai(test_dataset, te3_openai)

100%|██████████| 100/100 [00:17<00:00,  5.88it/s]


In [38]:
te3_results_df = pd.DataFrame(te3_results)

In [39]:
te3_hit_rate = te3_results_df["is_hit"].mean()
te3_hit_rate

0.98

### `Snowflake/snowflake-arctic-embed-m` (base)

In [40]:
from langchain_huggingface import HuggingFaceEmbeddings

huggingface_embeddings = HuggingFaceEmbeddings(model_name="Snowflake/snowflake-arctic-embed-m")
arctic_embed_m_results = evaluate_openai(test_dataset, huggingface_embeddings)

100%|██████████| 100/100 [00:03<00:00, 32.72it/s]


In [41]:
arctic_embed_m_results_df = pd.DataFrame(arctic_embed_m_results)

In [42]:
arctic_embed_m_hit_rate = arctic_embed_m_results_df["is_hit"].mean()
arctic_embed_m_hit_rate

0.58

### `Snowflake/snowflake-arctic-embed-m` (fine-tuned)

In [43]:
finetune_embeddings = HuggingFaceEmbeddings(model_name="finetuned_arctic")
finetune_results = evaluate_openai(test_dataset, finetune_embeddings)

Some weights of BertModel were not initialized from the model checkpoint at finetuned_arctic and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 100/100 [00:01<00:00, 59.87it/s]


In [44]:
finetune_results_df = pd.DataFrame(finetune_results)

In [45]:
finetune_hit_rate = finetune_results_df["is_hit"].mean()
finetune_hit_rate

1.0

# 🤝 Breakout Room #2

## Task 1: Vibe Checking the RAG Pipeline

We're going to use our RAG pipeline to vibe check on some common phrases now that we've modified it!

### Creating New Chunks

In order to try and evaluate our system more fairly, let's create new chunks that we will use to create our Vector Store.

In [46]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap  = 50,
    length_function = len
)

training_documents = text_splitter.split_documents(training_documents_loaded.load())

### Base Chain

We'll start by constructing our base chain, which will use the untrained retrieval model.

#### R - Retrieval

In [47]:
from langchain_community.vectorstores import FAISS

base_vectorstore = FAISS.from_documents(training_documents, huggingface_embeddings)
base_retriever = base_vectorstore.as_retriever(search_kwargs={"k": 6})

#### A - Augmented

In [48]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and a question, you must answer the question. If you do not know the answer, you must state that you do not know.

Context:
{context}

Question:
{question}

Answer:
"""

rag_prompt_template = ChatPromptTemplate.from_template(RAG_PROMPT)

#### G - Generation

In [49]:
rag_llm =  ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

#### RAG - LCEL RAG Pipeline

In [50]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

base_rag_chain = (
    {"context": itemgetter("question") | base_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt_template | rag_llm | StrOutputParser(), "context": itemgetter("context")}
)

In [51]:
base_rag_chain.invoke({"question" : "Why does the EU want to regulate AI?"})["response"]

'The EU wants to regulate AI to promote a human-centric approach to AI, ensure the development of secure, trustworthy, and ethical AI, protect ethical principles, and facilitate the protection of natural persons, democracy, the rule of law, and environmental protection. Additionally, the regulation aims to boost innovation and employment, positioning the Union as a leader in the uptake of trustworthy AI.'

In [52]:
base_rag_chain.invoke({"question" : "What are the codes of practice?"})["response"]

'I do not know.'

In [53]:
base_rag_chain.invoke({"question" : "How many parameters is too many parameters?"})["response"]

'The context suggests that models with at least a billion parameters are considered to display significant generality and competence in performing a wide range of tasks. Therefore, a model with fewer than a billion parameters may not be regarded as having "too many" parameters in this context. However, the specific threshold for "too many" parameters is not explicitly defined.'

In [54]:
base_rag_chain.invoke({"question" : "What is an emotion recognition system and why is it important?"})["response"]

'An emotion recognition system is a type of artificial intelligence (AI) technology designed to identify and interpret human emotions based on various inputs, such as facial expressions, voice tone, body language, or biometric data. These systems analyze patterns in the data to infer the emotional state of an individual.\n\nThe importance of emotion recognition systems lies in their potential applications across various fields, including mental health, customer service, security, and human-computer interaction. They can enhance user experiences, improve communication, and provide insights into emotional well-being. However, there are significant concerns regarding their reliability, specificity, and potential for discriminatory outcomes, particularly given the variability of emotional expression across different cultures and individuals.'

### Fine-tuned Embedding Model

Now let's rebuild our RAG chain with the Fine-tuned model - the only component we need to change is our `FAISS` vectorstore!

In [55]:
finetune_vectorstore = FAISS.from_documents(training_documents, finetune_embeddings)
finetune_retriever = finetune_vectorstore.as_retriever(search_kwargs={"k": 6})

In [56]:
finetune_rag_chain = (
    {"context": itemgetter("question") | finetune_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt_template | rag_llm | StrOutputParser(), "context": itemgetter("context")}
)

In [57]:
finetune_rag_chain.invoke({"question" : "Why does the EU want to regulate AI?"})["response"]

'The EU wants to regulate AI to improve the functioning of the internal market, promote the uptake of human-centric and trustworthy artificial intelligence, ensure a high level of protection of health, safety, and fundamental rights, protect against the harmful effects of AI systems, and support innovation. The regulation aims to create a uniform legal framework that prevents Member States from imposing restrictions on the development, marketing, and use of AI systems, unless explicitly authorized by the regulation.'

In [58]:
finetune_rag_chain.invoke({"question" : "What are the codes of practice?"})["response"]

'Codes of practice are guidelines developed to ensure compliance with obligations under regulations concerning general-purpose AI models. They are intended to cover obligations for providers of these models, particularly those that present systemic risks. The codes aim to establish a risk taxonomy for systemic risks at the Union level, including their sources, and focus on specific risk assessment and mitigation measures. The AI Office is responsible for encouraging, facilitating, and reviewing these codes, which should reflect the state of the art and consider diverse perspectives. The codes of practice are expected to be ready by May 2, 2025.'

In [59]:
finetune_rag_chain.invoke({"question" : "How many parameters is too many parameters?"})["response"]

'The context suggests that models with at least a billion parameters are considered to display significant generality and can competently perform a wide range of distinctive tasks. Therefore, it implies that having a billion parameters is a threshold for significant generality, but it does not specify a limit for "too many" parameters. Thus, I do not know how many parameters would be considered "too many."'

In [60]:
finetune_rag_chain.invoke({"question" : "What is an emotion recognition system and why is it important?"})["response"]

'An emotion recognition system is defined as an AI system that identifies or infers the emotions or intentions of natural persons based on their biometric data. This includes emotions such as happiness, sadness, anger, surprise, disgust, embarrassment, excitement, shame, contempt, satisfaction, and amusement. It does not encompass physical states like pain or fatigue.\n\nThe importance of emotion recognition systems lies in their potential applications across various fields, such as enhancing user experience in technology, improving mental health assessments, and aiding in security and surveillance. However, there are significant concerns regarding their reliability, cultural variability in emotional expression, and the potential for discriminatory outcomes, which raises ethical considerations about their use and impact on individual rights and freedoms.'

#####❓Question #2:

Which LCEL RAG Chain do you think answered the questions better, and why?

## Task 2: RAGAS Evaluation

It's great to have some idea of how our system is doing based on vibe-checks, but let's use RAGAS to provide more insight info. on how things are improving!

In [61]:
!pip install -qU ragas

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### RAGAS Synthetic Testset Generation

First things first, we need to generate some data to test our model on.

Let's use our test data that we created before as a base!

In [62]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
critic_llm = ChatOpenAI(model="gpt-4o-mini")
embeddings = OpenAIEmbeddings()

In [63]:
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

In [64]:
testset = generator.generate_with_langchain_docs(test_split_documents, test_size=20, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})

Filename and doc_id are the same for all nodes.                  
Generating: 100%|██████████| 20/20 [00:30<00:00,  1.54s/it]


In [65]:
testset.to_pandas().head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What rules and requirements apply to regulated...,[(158) Union financial services law includes i...,Internal governance and risk-management rules ...,simple,"[{'source': 'eu_ai_act.html', 'id': '6dff8f26-...",True
1,What is the purpose of the AI Office in handli...,"[the AI Office, it should provide for the poss...",The purpose of the AI Office in handling compl...,simple,"[{'source': 'eu_ai_act.html', 'id': '7d75291d-...",True
2,How does the European Central Bank collaborate...,"[as appropriate, into their existing superviso...",The European Central Bank collaborates with na...,simple,"[{'source': 'eu_ai_act.html', 'id': '4d20de2d-...",True
3,How can market surveillance authorities cooper...,"[one purpose that is classified as high-risk, ...",Market surveillance authorities can cooperate ...,simple,"[{'source': 'eu_ai_act.html', 'id': '0aa76e31-...",True
4,What role do governance mechanisms play in fos...,[(165) The development of AI systems other tha...,Governance mechanisms play a crucial role in f...,simple,"[{'source': 'eu_ai_act.html', 'id': '905efe30-...",True


### Generating Answer Datasets

For each of our pipelines, let's generate answers to these questions!

Once we have our: Questions, Answers, Contexts, Ground Truths we can move on to evaluating our datasets!

In [66]:
from datasets import Dataset

def generate_answers(chain, testset):
  answers = []
  contexts = []
  questions = testset.to_pandas()["question"].values.tolist()
  ground_truths = testset.to_pandas()["ground_truth"].values.tolist()

  for question in tqdm(questions):
    answer = chain.invoke({"question" : question})
    answers.append(answer["response"])
    contexts.append([context.page_content for context in answer["context"]])

  return Dataset.from_dict({
      "question" : questions,
      "answer" : answers,
      "contexts" : contexts,
      "ground_truth" : ground_truths
  })

In [67]:
base_dataset = generate_answers(base_rag_chain, testset)

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:30<00:00,  1.53s/it]


In [68]:
finetune_dataset = generate_answers(finetune_rag_chain, testset)

100%|██████████| 20/20 [00:27<00:00,  1.37s/it]


### Evaluating Using the Test Set

Now that we have a test set - it's time to evaluate our pipelines with it!

In [69]:
from ragas.metrics import (
    context_recall,
    context_precision,
)

In [70]:
from ragas import evaluate

result = evaluate(
    base_dataset,
    metrics=[
        context_precision,
        context_recall,
    ],
)

Evaluating: 100%|██████████| 40/40 [00:13<00:00,  2.89it/s]


In [71]:
result

{'context_precision': 0.6834, 'context_recall': 0.3958}

In [72]:
result.to_pandas().head()

,question,contexts,answer,ground_truth,context_precision,context_recall
0,What rules and requirements apply to regulated...,[(23) This Regulation should also apply to Uni...,I do not know.,Internal governance and risk-management rules ...,0.700000,0.000000
1,What is the purpose of the AI Office in handli...,"[the results of its monitoring activities, or ...",The purpose of the AI Office in handling compl...,The purpose of the AI Office in handling compl...,1.000000,0.500000
2,How does the European Central Bank collaborate...,[to the designation of national competent auth...,The European Central Bank collaborates with na...,The European Central Bank collaborates with na...,0.966667,1.000000
3,How can market surveillance authorities cooper...,[of protection or the need for compliance with...,Market surveillance authorities can cooperate ...,Market surveillance authorities can cooperate ...,0.444444,0.666667
4,What role do governance mechanisms play in fos...,"[therein, the mandatory requirements for high-...",The context provided does not explicitly menti...,Governance mechanisms play a crucial role in f...,0.366667,0.333333


In [73]:
result = evaluate(
    finetune_dataset,
    metrics=[
        context_precision,
        context_recall,
    ],
)

Evaluating: 100%|██████████| 40/40 [00:22<00:00,  1.81it/s]


In [74]:
result

{'context_precision': 0.9578, 'context_recall': 0.9125}

In [75]:
result.to_pandas().head()

,question,contexts,answer,ground_truth,context_precision,context_recall
0,What rules and requirements apply to regulated...,[(158) Union financial services law includes i...,Regulated financial institutions must adhere t...,Internal governance and risk-management rules ...,1.000000,1.000000
1,What is the purpose of the AI Office in handli...,[(164) The AI Office should be able to take th...,The purpose of the AI Office in handling compl...,The purpose of the AI Office in handling compl...,1.000000,0.500000
2,How does the European Central Bank collaborate...,"[authorities under this Regulation, the nation...",The European Central Bank collaborates with na...,The European Central Bank collaborates with na...,1.000000,1.000000
3,How can market surveillance authorities cooper...,[surveillance authorities remain responsible f...,Market surveillance authorities can cooperate ...,Market surveillance authorities can cooperate ...,1.000000,0.666667
4,What role do governance mechanisms play in fos...,[(165) The development of AI systems other tha...,Governance mechanisms play a crucial role in f...,Governance mechanisms play a crucial role in f...,0.926667,1.000000


#### 🏗️ Activity #3:

Discuss changes that you'd make to this pipeline based on the performance improvements that you see with RAGAS and the fine-tuning.

Come up with 3 changes, and then we'll discuss these options as a group!

**ANSWER**

Experiment with dfferent embedding models
Use a more advanced chunking strategy like semantic chunking.
Use advanced retriever techniques like context compression retriever and multi-query retriever